In [ ]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

# Import class Clustering từ cluster.py
from cluster import Clustering


In [ ]:

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    if abs(cos_beta) < 1e-9:
        return v_AUV
    return abs(math.cos(angle) * v_AUV / cos_beta)

def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    if len(path) <= 1:
        return 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # return to start
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


In [ ]:
def compute_energy(best_time, n_members):
    """
    Tính năng lượng tiêu thụ cho Member Node và Cluster Head.
    
    Parameters:
    - best_time: Thời gian hoàn thành chu kỳ AUV
    - n_members: Số lượng node thành viên thực tế trong cluster (không tính cluster head)
    """
    G, L = 100, 1024
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6

    # Năng lượng cho Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    # Năng lượng cho Cluster Head (nhận từ n_members node, truyền cho AUV)
    E_rx_TN = G * P_r * L * n_members / DR
    E_tx_TN = G * P_t * L * n_members / DR_i
    E_idle_TN = (best_time - (G*L*n_members/DR) - (G*L*n_members/DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_total": E_total_MN},
        "Target": {"E_total": E_total_TN}
    }

def update_energy(all_nodes, clusters, best_time):
    """
    Cập nhật năng lượng cho tất cả các node dựa trên số member thực tế của từng cluster.
    
    Parameters:
    - all_nodes: Dictionary chứa thông tin tất cả các node
    - clusters: Dictionary chứa thông tin các cluster
    - best_time: Thời gian hoàn thành chu kỳ AUV
    """
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        
        # Tính số member nodes (không tính cluster head)
        n_members = len([n for n in nodes if n != ch])
        
        # Tính năng lượng cho cluster này với số member thực tế
        energy_report = compute_energy(best_time, n_members)
        
        for nid in nodes:
            if nid not in all_nodes: continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)

def remove_dead_nodes(all_nodes, clusters):
    """
    Loại bỏ các node đã hết năng lượng và cập nhật lại clusters.
    
    Returns:
    - new_clusters: Dictionary các cluster còn node sống
    - dead: List các node_id đã chết
    """
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]

    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c

    return new_clusters, dead

def nearest_neighbor_path(centers):
    """
    Tìm đường đi ngắn nhất bằng thuật toán Nearest Neighbor (Greedy TSP).
    
    Parameters:
    - centers: List các tọa độ cần thăm (bắt đầu từ base station ở index 0)
    
    Returns:
    - path: List các index theo thứ tự thăm
    """
    n = len(centers)
    if n == 1:
        return [0]
    unvisited = set(range(1, n))
    path = [0]
    current = 0
    coords = np.array(centers)

    while unvisited:
        candidates = sorted(unvisited)
        cand_coords = coords[candidates]
        dists = np.linalg.norm(cand_coords - coords[current], axis=1)
        next_idx = candidates[int(np.argmin(dists))]
        path.append(next_idx)
        unvisited.remove(next_idx)
        current = next_idx

    return path

def recluster(all_nodes, node_positions, clustering_instance, r_sen=60, max_size=20, min_size=5):
    """
    Phân cụm lại toàn bộ các node còn sống sử dụng thuật toán từ cluster.py.
    
    Parameters:
    - all_nodes: Dictionary các node còn sống
    - node_positions: Dictionary vị trí của các node
    - clustering_instance: Instance của class Clustering
    - r_sen: Ngưỡng khoảng cách tối đa trong cụm
    - max_size: Số lượng node tối đa trong 1 cụm
    - min_size: Số lượng node tối thiểu trong 1 cụm
    
    Returns:
    - clusters: Dictionary các cụm mới
    """
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}

    # Tạo mảng tọa độ nodes
    coords = np.array([node_positions[nid] for nid in ids])
    
    # Cập nhật tham số cho clustering instance
    clustering_instance.r_sen = r_sen
    clustering_instance.max_cluster_size = max_size
    clustering_instance.min_cluster_size = min_size
    
    # Phân cụm với ràng buộc
    clusters_data = clustering_instance.cluster_with_constraints(coords, ids)
    
    # Chuyển đổi sang format cần thiết
    clusters = {}
    for i, (cluster_nodes, cluster_ids) in enumerate(clusters_data):
        center = np.mean(cluster_nodes, axis=0).tolist()
        
        # Chọn cluster head
        ch = clustering_instance.choose_cluster_head(cluster_nodes, cluster_ids, all_nodes)
        
        clusters[i] = {
            'nodes': cluster_ids,
            'center': center,
            'cluster_head': ch
        }
    
    return clusters


In [ ]:

def main():
    """
    Hàm chính mô phỏng mạng cảm biến dưới nước với AUV thu thập dữ liệu.
    """
    input_dir = "input_data"
    output_dir = "result_ga_ch_most_energy"
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_dir):
        print(f"❌ Lỗi: Thư mục {input_dir} không tồn tại!")
        return

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    
    if len(files) == 0:
        print(f"❌ Không tìm thấy file dữ liệu nào trong {input_dir}")
        return

    # Tham số
    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0
    R_SEN = 60      # Bán kính truyền tải (m)
    MAX_SIZE = 20   # Kích thước cụm tối đa
    MIN_SIZE = 5    # Kích thước cụm tối thiểu
    
    results_summary = []
    
    # Khởi tạo Clustering instance
    clustering = Clustering(
        space_size=400,
        r_sen=R_SEN,
        max_cluster_size=MAX_SIZE,
        min_cluster_size=MIN_SIZE
    )

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n{'='*60}")
        print(f"=== Đang xử lý file: {filename} ===")
        print(f"{'='*60}")
        
        try:
            with open(input_path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc file {filename}: {e}")
            continue

        node_positions = {}
        all_nodes = {}
        
        # Xử lý file JSON - danh sách nodes: [{"id": 0, "x": ..., "y": ..., "z": ...}, ...]
        if isinstance(data, list):
            print("Định dạng: Danh sách nodes")
            for node in data:
                nid = node['id']
                all_nodes[nid] = {
                    'initial_energy': node.get('initial_energy', INITIAL_ENERGY),
                    'residual_energy': node.get('residual_energy', INITIAL_ENERGY)
                }
                node_positions[nid] = (node['x'], node['y'], node['z'])
        else:
            print(f"❌ Cấu trúc file {filename} không được hỗ trợ (cần list of nodes)")
            continue

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")
        print(f"Tham số: r_sen={R_SEN}m, max_size={MAX_SIZE}, min_size={MIN_SIZE}")

        # Phân cụm lần đầu tiên
        print(f"\n{'='*60}")
        print("PHÂN CỤM LẦN ĐẦU TIÊN")
        print(f"{'='*60}")
        initial_clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)
        
        # Tính metrics cho phân cụm ban đầu
        clusters_data_for_metrics = []
        for cid, cinfo in initial_clusters.items():
            cluster_nodes = np.array([node_positions[nid] for nid in cinfo['nodes']])
            clusters_data_for_metrics.append((cluster_nodes, cinfo['nodes']))
        
        metrics = clustering.calculate_metrics(clusters_data_for_metrics)
        
        print(f"\n METRICS PHÂN CỤM:")
        print(f"  - Số cụm: {metrics['num_clusters']}")
        print(f"  - Kích thước TB: {metrics['avg_cluster_size']:.1f}")
        print(f"  - Kích thước: [{metrics['min_cluster_size']} - {metrics['max_cluster_size']}]")
        print(f"  - Khoảng cách TB trong cụm: {metrics['avg_intra_distance']:.1f}m")
        print(f"  - Khoảng cách max trong cụm: {metrics['max_intra_distance']:.1f}m")
        print(f"  - Độ cân bằng: {metrics['balance_score']:.2%}")
        
        # Lưu kết quả phân cụm lần đầu
        clusters_output = {}
        for cid, cinfo in initial_clusters.items():
            clusters_output[cid] = {
                'cluster_id': cid,
                'nodes': cinfo['nodes'],
                'center': cinfo['center'],
                'cluster_head': cinfo['cluster_head'],
                'num_nodes': len(cinfo['nodes'])
            }
        
        initial_cluster_file = os.path.join(output_dir, f"initial_clusters_{filename}")
        with open(initial_cluster_file, "w", encoding='utf-8') as f:
            json.dump(clusters_output, f, indent=4, ensure_ascii=False)
        
        print(f"\n Đã lưu phân cụm ban đầu: {initial_cluster_file}")

        cycle = 0
        alive_log = []
        energy_log = []
        cluster_count_log = []

        # Vòng lặp mô phỏng
        print(f"\n{'='*60}")
        print(" BẮT ĐẦU MÔ PHỎNG")
        print(f"{'='*60}")
        
        while True:
            cycle += 1
            alive_log.append(len(all_nodes))
            total_energy = sum(all_nodes[n]['residual_energy'] for n in all_nodes)
            energy_log.append(total_energy)

            alive_ratio = len(all_nodes)/total_nodes if total_nodes > 0 else 0
            
            if alive_ratio < 0.9:
                print(f"\n🛑 Dừng mô phỏng ở cycle {cycle}: < 90% node còn sống ({alive_ratio*100:.2f}%)")
                break

            # Phân cụm lại
            clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)
            cluster_count_log.append(len(clusters))
            
            if len(clusters) == 0:
                print(f"\n Dừng mô phỏng ở cycle {cycle}: Không còn node")
                break

            print(f"\n--- Cycle {cycle} --- | Alive: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes}) | Energy: {total_energy:.2f}J | Clusters: {len(clusters)}")

            # Tạo đường đi cho AUV
            sorted_keys = sorted(clusters.keys())
            centers = [(0, 0, 0)]  # Base station
            for k in sorted_keys:
                centers.append(tuple(clusters[k]['center']))

            path_indices = nearest_neighbor_path(centers)
            best_time = travel_time(path_indices, centers, v_f, v_AUV)
            
            # Cập nhật năng lượng
            update_energy(all_nodes, clusters, best_time)
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            
            if dead_nodes:
                print(f"   ⚡ {len(dead_nodes)} node(s) đã hết năng lượng")

        # Lưu kết quả JSON
        meta = {
            'input_file': filename,
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'final_alive_nodes': len(all_nodes),
            'final_alive_ratio': len(all_nodes)/total_nodes if total_nodes > 0 else 0,
            'parameters': {
                'r_sen': R_SEN,
                'max_cluster_size': MAX_SIZE,
                'min_cluster_size': MIN_SIZE,
                'v_flow': v_f,
                'v_AUV': v_AUV
            },
            'initial_clustering_metrics': {
                'num_clusters': metrics['num_clusters'],
                'avg_cluster_size': float(metrics['avg_cluster_size']),
                'balance_score': float(metrics['balance_score'])
            }
        }
        
        output_json = os.path.join(output_dir, f"result_{filename}")
        with open(output_json, "w") as f:
            json.dump(meta, f, indent=4)

        results_summary.append((filename, cycle - 1))
        print(f"\n File {filename}: {cycle - 1} cycles hoàn thành")

        # Plot 1: Alive nodes per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(alive_log)), alive_log, marker='o', linewidth=2, color='steelblue')
        plt.title(f"Số node sống theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Nodes alive", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.axhline(y=total_nodes*0.9, color='red', linestyle='--', linewidth=2, label='Ngưỡng 90%')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"alive_nodes_{filename}.png"), dpi=150)
        plt.close()

        # Plot 2: Total energy per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(energy_log)), energy_log, marker='s', linewidth=2, color='orange')
        plt.title(f"Năng lượng toàn mạng theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Total energy (J)", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"energy_{filename}.png"), dpi=150)
        plt.close()
        
        # Plot 3: Number of clusters per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(cluster_count_log)+1), cluster_count_log, marker='^', linewidth=2, color='green')
        plt.title(f"Số cụm theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Số cụm", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"clusters_{filename}.png"), dpi=150)
        plt.close()

    # Summary plot
    if results_summary:
        labels = [x[0] for x in results_summary]
        values = [x[1] for x in results_summary]
        plt.figure(figsize=(10, 6))
        plt.plot(labels, values, marker='o', linewidth=2, markersize=8)
        plt.title("AUV cycles completed per dataset", fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel("Cycles", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "summary_cycles.png"), dpi=150)
        plt.close()
        
        print(f"\n{'='*60}")
        print(f"✅ Hoàn thành! Kết quả đã lưu tại: {output_dir}")
        print(f"{'='*60}")
    else:
        print("\n  Không có kết quả nào được tạo ra")


In [ ]:
if __name__ == '__main__':
    main()
